In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from eipy.ei import EnsembleIntegration
import eipy.utils as ut
from eipy.additional_ensembles import MeanAggregation, CES
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn import datasets
pd.set_option('display.max_columns', None)

In [2]:
# If data is multi-class, run a check on the allowable base and meta models.

base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(multi_class="multinomial"),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [3]:

"""
For filtering base predictors by whether or not they rely on heursitics for multiclass extension

natively_multi_class_predictors = ["XGBClassifier",
"BernoulliNB",
"DecisionTreeClassifier",
"ExtraTreeClassifier",
"GaussianNB",
"KNeighborsClassifier",
"LabelPropagation",
"LabelSpreading",
"LinearDiscriminantAnalysis",
"LinearSVC", #(setting multi_class=”crammer_singer”)
"LogisticRegression", #(setting multi_class=”multinomial”)
"LogisticRegressionCV", #(setting multi_class=”multinomial”)
"MLPClassifier",
"NearestCentroid",
"QuadraticDiscriminantAnalysis",
"RadiusNeighborsClassifier",
"RandomForestClassifier",
"RidgeClassifier",
"RidgeClassifierCV"]

base_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}
"""

'\nFor filtering base predictors by whether or not they rely on heursitics for multiclass extension\n\nnatively_multi_class_predictors = ["XGBClassifier",\n"BernoulliNB",\n"DecisionTreeClassifier",\n"ExtraTreeClassifier",\n"GaussianNB",\n"KNeighborsClassifier",\n"LabelPropagation",\n"LabelSpreading",\n"LinearDiscriminantAnalysis",\n"LinearSVC", #(setting multi_class=”crammer_singer”)\n"LogisticRegression", #(setting multi_class=”multinomial”)\n"LogisticRegressionCV", #(setting multi_class=”multinomial”)\n"MLPClassifier",\n"NearestCentroid",\n"QuadraticDiscriminantAnalysis",\n"RadiusNeighborsClassifier",\n"RandomForestClassifier",\n"RidgeClassifier",\n"RidgeClassifierCV"]\n\nbase_predictors = {k : v for k,v in base_predictors.items() if str(v).split("(")[0] in natively_multi_class_predictors}\n'

In [4]:
"""https://dev.pages.lis-lab.fr/scikit-multimodallearn/tutorial/auto_examples/combo/plot_combo_3_views_3_classes.html#
multi modal multi-class toy data generation"""

def generate_data(n_samples, lim):
    """Generate random data in a rectangle"""
    lim = np.array(lim)
    n_features = lim.shape[0]
    data = np.random.random((n_samples, n_features))
    data = (lim[:, 1]-lim[:, 0]) * data + lim[:, 0]
    return data
seed = 12
np.random.seed(seed)

n_samples = 300

modality_0 = np.concatenate((generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]])))

modality_1 = np.concatenate((generate_data(n_samples, [[1., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]])))

modality_2 = np.concatenate((generate_data(n_samples, [[0., 2.], [0., 1.]]),
                         generate_data(n_samples, [[0., 1.], [0., 1.]]),
                         generate_data(n_samples, [[1., 2.], [0., 1.]])))

y = np.zeros(3*n_samples, dtype=np.int64)
y[n_samples:2*n_samples] = 1
y[2*n_samples:] = 2


In [5]:
X_0_train, X_0_test, y_train, y_test = train_test_split(modality_0, y, test_size=0.2, random_state=3, stratify=y)
X_1_train, X_1_test, _,_ = train_test_split(modality_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _,_ = train_test_split(modality_2, y, test_size=0.2, random_state=3, stratify=y)

In [6]:
data_train = {
                "Modality_0": X_0_train,
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_0": X_0_test,
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

In [7]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=42,
                        project_name="toy",
                        model_building=True,
                        )



In [8]:
EI.train_base(data_train, y_train)

Training base predictors on Modality_0...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_1...
        
... for ensemble performance analysis...


...

Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_2...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


In [9]:
EI.meta_training_data[0]

modality          Modality_0                                              \
base predictor          ADAB                               XGB             
sample                     0                                 0             
class                      0             1         2         0         1   
0               5.033375e-01  2.222770e-16  0.496663  0.990476  0.000190   
1               5.033375e-01  2.222770e-16  0.496663  0.082868  0.000607   
2               2.223397e-16  5.036501e-01  0.496350  0.008628  0.492612   
3               2.223397e-16  5.036501e-01  0.496350  0.000314  0.914500   
4               5.033375e-01  2.222770e-16  0.496663  0.205049  0.002299   
..                       ...           ...       ...       ...       ...   
571             5.034012e-01  2.222858e-16  0.496599  0.730658  0.000891   
572             5.034012e-01  2.222858e-16  0.496599  0.741070  0.003562   
573             2.223192e-16  5.035152e-01  0.496485  0.001255  0.920323   
574             5.034012e-01  2.222858e-16  0.496599  0.988050  0.000777   
575             2.223192e-16  5.035152e-01  0.496485  0.000728  0.915664   

modality                                                                       \
base predictor             DT              RF                    GB             
sample                      0               0                     0             
class                  2    0    1    2     0     1     2         0         1   
0               0.009334  1.0  0.0  0.0  0.91  0.00  0.09  0.873335  0.006062   
1               0.916525  0.0  0.0  1.0  0.54  0.00  0.46  0.305751  0.005220   
2               0.498759  0.0  0.0  1.0  0.04  0.63  0.33  0.012749  0.491289   
3               0.085186  0.0  1.0  0.0  0.01  0.74  0.25  0.007984  0.813579   
4               0.792652  1.0  0.0  0.0  0.52  0.01  0.47  0.495245  0.010051   
..                   ...  ...  ...  ...   ...   ...   ...       ...       ...   
571             0.268450  1.0  0.0  0.0  0.85  0.00  0.15  0.326098  0.010482   
572             0.255368  0.0  0.0  1.0  0.73  0.00  0.27  0.758719  0.006605   
573             0.078422  0.0  1.0  0.0  0.00  0.74  0.26  0.003857  0.919627   
574             0.011173  1.0  0.0  0.0  0.87  0.00  0.13  0.924327  0.001435   
575             0.083608  0.0  1.0  0.0  0.00  0.83  0.17  0.005214  0.853261   

modality                                                               \
base predictor            KNN                  LR                       
sample                      0                   0                       
class                  2    0    1    2         0         1         2   
0               0.120604  1.0  0.0  0.0  0.741568  0.022422  0.236010   
1               0.689029  0.4  0.0  0.6  0.688637  0.025537  0.285826   
2               0.495962  0.2  0.4  0.4  0.260780  0.271721  0.467498   
3               0.178437  0.4  0.4  0.2  0.217677  0.251799  0.530523   
4               0.494704  0.4  0.0  0.6  0.389640  0.149040  0.461319   
..                   ...  ...  ...  ...       ...       ...       ...   
571             0.663420  0.8  0.0  0.2  0.813545  0.010634  0.175820   
572             0.234676  0.8  0.0  0.2  0.491131  0.072969  0.435900   
573             0.076515  0.0  0.8  0.2  0.025359  0.779247  0.195393   
574             0.074238  0.8  0.0  0.2  0.761829  0.009976  0.228195   
575             0.141524  0.0  0.8  0.2  0.019398  0.783810  0.196792   

modality                                                                    \
base predictor        NB                           MLP                       
sample                 0                             0                       
class                  0         1         2         0         1         2   
0               0.787548  0.000082  0.212370  0.734722  0.004587  0.260691   
1               0.743124  0.000103  0.256774  0.661490  0.010317  0.328193   
2               0.197136  0.270495  0.532369  0.315407  0.241999  0.442594   
3

In [10]:
EI.meta_test_data[0]

modality          Modality_0                                              \
base predictor          ADAB                               XGB             
sample                     0                                 0             
class                      0             1         2         0         1   
0               2.223189e-16  5.035294e-01  0.496471  0.000618  0.720623   
1               5.034139e-01  2.222883e-16  0.496586  0.219468  0.005430   
2               5.034139e-01  2.222883e-16  0.496586  0.732608  0.000160   
3               5.034139e-01  2.222883e-16  0.496586  0.876348  0.000254   
4               5.034139e-01  2.222883e-16  0.496586  0.492079  0.000627   
..                       ...           ...       ...       ...       ...   
139             5.034139e-01  2.222883e-16  0.496586  0.598946  0.001246   
140             5.034139e-01  2.222883e-16  0.496586  0.895722  0.000379   
141             2.223189e-16  5.035294e-01  0.496471  0.000606  0.628569   
142             2.223189e-16  5.035294e-01  0.496471  0.000275  0.901953   
143             2.223189e-16  5.035294e-01  0.496471  0.000562  0.853588   

modality                                                                       \
base predictor             DT              RF                    GB             
sample                      0               0                     0             
class                  2    0    1    2     0     1     2         0         1   
0               0.278758  0.0  1.0  0.0  0.00  0.83  0.17  0.007180  0.731360   
1               0.775102  0.0  0.0  1.0  0.24  0.00  0.76  0.534920  0.008563   
2               0.267231  1.0  0.0  0.0  0.70  0.00  0.30  0.707335  0.006983   
3               0.123398  1.0  0.0  0.0  0.76  0.00  0.24  0.850283  0.002924   
4               0.507294  0.0  0.0  1.0  0.63  0.00  0.37  0.272582  0.009598   
..                   ...  ...  ...  ...   ...   ...   ...       ...       ...   
139             0.399808  1.0  0.0  0.0  0.65  0.01  0.34  0.562459  0.007929   
140             0.103899  1.0  0.0  0.0  0.72  0.00  0.28  0.676814  0.001660   
141             0.370825  0.0  1.0  0.0  0.00  0.74  0.26  0.005543  0.713472   
142             0.097772  0.0  1.0  0.0  0.01  0.75  0.24  0.005624  0.654730   
143             0.145850  0.0  1.0  0.0  0.00  0.53  0.47  0.008667  0.618650   

modality                                                               \
base predictor            KNN                  LR                       
sample                      0                   0                       
class                  2    0    1    2         0         1         2   
0               0.261460  0.0  0.8  0.2  0.015874  0.766577  0.217548   
1               0.456517  0.6  0.0  0.4  0.445201  0.090013  0.464786   
2               0.285682  0.8  0.0  0.2  0.801036  0.014812  0.184152   
3               0.146793  0.6  0.0  0.4  0.260913  0.198842  0.540244   
4               0.717821  0.8  0.0  0.2  0.805281  0.012175  0.182545   
..                   ...  ...  ...  ...       ...       ...       ...   
139             0.429612  0.4  0.0  0.6  0.660310  0.035683  0.304007   
140             0.321527  0.6  0.0  0.4  0.721730  0.014693  0.263578   
141             0.280984  0.0  0.8  0.2  0.190736  0.273824  0.535441   
142             0.339645  0.0  0.8  0.2  0.007671  0.832865  0.159464   
143             0.372683  0.0  0.6  0.4  0.025428  0.665495  0.309077   

modality                                                                    \
base predictor        NB                           MLP                       
sample                 0                             0                       
class                  0         1         2         0         1         2   
0               0.000027  0.769516  0.230457  0.003011  0.728071  0.268918   
1               0.622085  0.008826  0.369089  0.555652  0.052631  0.391717   
2               0.780021  0.000052  0.219927  0.726409  0.002624  0.270968   
3

In [11]:
# create a summary of base predictor performance
EI.base_summary = ut.create_base_summary(EI.meta_test_data)
EI.base_summary

     (Modality_0, ADAB, 0)  (Modality_0, DT, 0)  (Modality_0, GB, 0)  \
0                        1                    1                    1   
1                        0                    2                    0   
2                        0                    0                    0   
3                        0                    0                    0   
4                        0                    2                    2   
..                     ...                  ...                  ...   
139                      0                    0                    0   
140                      0                    0                    0   
141                      1                    1                    1   
142                      1                    1                    1   
143                      1                    1                    1   

     (Modality_0, KNN, 0)  (Modality_0, LR, 0)  (Modality_0, MLP, 0)  \
0                       1                    1                 

AssertionError: axis must be a MultiIndex

In [ ]:
EI.train_meta(meta_predictors=base_predictors)

Analyzing ensembles: |          |  0%

Analyzing ensembles: |          |  0%


ValueError: Classification metrics can't handle a mix of multiclass and continuous-multioutput targets

In [ ]:
EI.meta_summary["metrics"]

In [ ]:
y_pred = EI.predict(X_dict=data_test, meta_model_key="XGB")
y_pred = np.round(y_pred)
y_pred

In [ ]:
y_test

In [ ]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred))])/len(y_test)
accuracy # =179/180

In [ ]:
EI.meta_summary['thresholds']["XGB"]

In [ ]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

Modality_a = X[:, 0:2]
Modality_b = X[:, 2:4]

X_a_train, X_a_test, y_train, y_test = train_test_split(Modality_a, y, test_size=0.2, random_state=3, stratify=y)
X_b_train, X_b_test, _,_ = train_test_split(Modality_b, y, test_size=0.2, random_state=3, stratify=y)

In [ ]:
iris_data_train = {
                "Modality_a": X_a_train,
                "Modality_b": X_b_train
                }

iris_data_test = {
                "Modality_a": X_a_test,
                "Modality_b": X_b_test
                }

In [ ]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(),
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True)
}

In [ ]:
EI_iris = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy=None,
                        n_jobs=-1,
                        random_state=0,
                        project_name="iris",
                        model_building=True,
                        )


In [ ]:
for name, modality in iris_data_train.items():
    EI_iris.train_base(modality, y_train, modality_name=name)

In [ ]:
EI_iris.meta_training_data

In [ ]:
EI_iris.train_meta(meta_predictors=base_predictors)

In [ ]:
EI_iris.meta_summary["metrics"]

In [ ]:
y_pred_iris = EI_iris.predict(X_dict=iris_data_test, meta_model_key="SVM")
y_pred_iris = np.round(y_pred_iris)
y_pred_iris

In [ ]:
accuracy = sum([1*(y==y_hat)+0*(y!=y_hat) for y,y_hat in list(zip(y_test, y_pred_iris))])/len(y_test)
accuracy

In [ ]:
import pandas as pd
import numpy as np

# Create a dictionary with 2D arrays as values
data = {
    'Column1': [np.array([1, 2]), np.array([3, 4]), np.array([5, 6])],
    'Column2': [np.array([10, 20]), np.array([30, 40]), np.array([50, 60])]
}

# Create a DataFrame with MultiIndex
df = pd.DataFrame()

# Iterate over the dictionary items and reshape 2D arrays into sub-columns
for key, value in data.items():
    num_cols = value[0].shape[0]  # Get the shape of the 2D array
    print(num_cols)
    for i in range(num_cols):
        # Create sub-columns for each element of the 2D array
        sub_column_name = f'SubColumn{i + 1}'
        df[(key, sub_column_name)] = [row[i] for row in value]

# Print the DataFrame
print(df)
